In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def exponential_rate(qi, Di, t):
    """
    Arps exponential decline:
        q(t) = qi * exp(-Di * t)
    Di: nominal decline rate (1/time)
    """
    t = np.asarray(t, dtype=float)
    qi = np.asarray(qi, dtype=float)
    Di = np.asarray(Di, dtype=float)
    if np.any(Di < 0):
        raise ValueError("Di must be >= 0")
    return qi * np.exp(-Di * t)

def hyperbolic_rate(qi, b, Di, t):
    """
    Arps hyperbolic decline:
        q(t) = qi * ((1 - Di) ** (b * t))
    b: decline acceleration (1/time)
    Di: nominal decline rate (1/time)
    """

    t = np.asarray(t, dtype=float)
    qi = np.asarray(qi, dtype=float)
    b = np.asarray(b, dtype=float)
    Di = np.asarray(Di, dtype=float)
    if np.any(Di < 0):
        raise ValueError("Di must be >= 0")
    if np.any(b < 0):
        raise ValueError("b must be >= 0")

    # Handle b=0 smoothly (limit -> exponential)
    eps = 1e-12
    if np.isscalar(b) and abs(b) < eps:
        return exponential_rate(qi, Di, t)

    # For array b, blend where b ~ 0
    out = np.empty_like(t, dtype=float)
    mask = np.abs(b) < eps
    if np.any(mask):
        out[mask] = exponential_rate(qi, Di, t[mask])
    if np.any(~mask):
        out[~mask] = qi / np.power(1.0 + b * Di * t[~mask], 1.0 / b)
    return out

def power_law_rate(qi, m, tau, t):
    """
    Power-law decline:
        q(t) = qi / (1 + t / tau) ** m

    m: dimensionless exponent (>= 0)
    tau: characteristic time scale (> 0, same units as t)
    """
    t = np.asarray(t, dtype=float)
    qi = np.asarray(qi, dtype=float)
    m = np.asarray(m, dtype=float)
    tau = np.asarray(tau, dtype=float)

    if np.any(m < 0):
        raise ValueError("m must be >= 0")
    if np.any(tau <= 0):
        raise ValueError("tau must be > 0")

    return qi / np.power(1.0 + (t / tau), m)

def linear_rate(qi, qf, t, t_end):
    """
    Linear rate interpolation from qi at t=0 to qf at t=t_end:
        q(t) = qi + (qf - qi) * (t / t_end)
    """
    t = np.asarray(t, dtype=float)
    qi = float(qi)
    qf = float(qf)
    t_end = float(t_end)
    if t_end <= 0:
        raise ValueError("t_end must be > 0")
    return qi + (qf - qi) * (t / t_end)


def flat_rate(qi, t, t_end):
    """
    Flat rate:
        q(t) = qi
    """
    t = np.asarray(t, dtype=float)
    return np.ones_like(t) * qi

In [ ]:
# Multi-segment decline sandbox
#
# This uses `playground/decline_multiseg.py` so it’s test-covered, but we convert
# to a DataFrame here for easy notebook exploration.

from pathlib import Path
import sys

# Ensure repo root is on sys.path (so `import playground...` works from notebooks)
for p in [Path.cwd(), *Path.cwd().parents]:
    if (p / "playground").exists():
        sys.path.insert(0, str(p))
        break

from playground.decline_multiseg import (
    SegmentSpec,
    simulate_multisegment,
    multisegment_to_dataframe,
)

# Example: 3 segments on a monthly grid (time in years)
segments = [
    SegmentSpec(method="Hyperbolic", duration=2.0, params={"qi": 1200.0, "b": 1.2, "Di": 1.5}),
    SegmentSpec(method="Exp", duration=3.0, params={"Di": 0.25}),
    SegmentSpec(method="Flat", duration=1.0, params={}),
]

out = simulate_multisegment(segments, dt_years=1.0 / 12.0)
df = multisegment_to_dataframe(out)

# Optional convenience metrics
# - cumulative % of final cumulative (helps visualize EUR build)
df["cum_pct_of_final"] = np.where(df["cum"].iloc[-1] != 0, 100.0 * df["cum"] / df["cum"].iloc[-1], 0.0)

df.head(12)

plt.figure(figsize=(12, 5))
sns.lineplot(data=df, x="t_years", y="rate", hue="segment", palette="tab10")
plt.title("Multi-segment decline rate")
plt.xlabel("Time (years)")
plt.ylabel("Rate")
plt.grid(True, alpha=0.25)
plt.show()

plt.figure(figsize=(12, 5))
sns.lineplot(data=df, x="t_years", y="secant_nominal_pct_per_year", hue="segment", palette="tab10")
plt.title("Secant nominal decline (pct/year)")
plt.xlabel("Time (years)")
plt.ylabel("Nominal decline (%/year)")
plt.grid(True, alpha=0.25)
plt.show()